## Second try

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import sentencepiece as spm
#import pandas as pd

import json


#from sklearn.model_selection import train_test_split


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Tokenizer

In [ ]:
texts = []
with open('train.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line.strip())
        texts.append(data['text'])

with open('corpus.txt', 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(text + '\n')


In [ ]:
spm.SentencePieceTrainer.train(
    input='corpus.txt',
    model_prefix='tokenizer',
    vocab_size=20000,
    model_type='unigram',
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    )

## Load tokenizer

In [2]:
sp = spm.SentencePieceProcessor()
sp.Load('tokenizer.model')  # Файл модели

True

##### TEST

In [ ]:
# Загрузка
sp = spm.SentencePieceProcessor()
sp.Load('tokenizer.model')  # Файл модели

# test
text = "Отличный русский текст для TextCNN классификации!"
tokens = sp.Encode(text, out_type=int)  # IDs
pieces = sp.Encode(text, out_type=str)  # Подслова

print("ОРИГИНАЛ:", text)
print("IDS:", tokens)
print("ТОКЕНЫ:", pieces)
print("Vocab size:", sp.GetPieceSize())
print("Длина:", len(tokens))


#### Data loader

In [3]:
class JsonLDataset(Dataset):
    def __init__(self, file_path, tokenizer_path='tokenizer.model', max_length=256):
        self.file_path = file_path
        self.max_length = max_length
        self.data = []

        self.sp = spm.SentencePieceProcessor()
        self.sp.load(tokenizer_path)

        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                sample = json.loads(line.strip())
                self.data.append({
                    'text': sample['text'],
                    'label': sample['label']
                })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        text = item['text']
        tokens = self.sp.encode(text, out_type=int)

        if len(tokens) > self.max_length:
            tokens = tokens[:self.max_length]
        else:
            pad_id = self.sp.PieceToId('<pad>')
            tokens = tokens + [pad_id] * (self.max_length - len(tokens))

        text_tensor = torch.tensor(tokens, dtype=torch.long)
        label_tensor = torch.tensor(item['label'], dtype=torch.long)

        return text_tensor, label_tensor

    

train_dataset = JsonLDataset('train.jsonl')
test_dataset = JsonLDataset('test.jsonl')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


## Model

In [4]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        # padding_idx=pad_idx говорит модели игнорировать нули (ускорение!)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        
        # Параллельные свертки (для 2, 3 и 4 слов)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim, 
                      out_channels=n_filters, 
                      kernel_size=fs)
            for fs in filter_sizes
        ])
        
        self.fc = nn.Linear(n_filters * len(filter_sizes), output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        # text: [batch, len] -> embedded: [batch, len, dim] -> [batch, dim, len]
        embedded = self.embedding(text).permute(0, 2, 1)
        
        # Свертка + ReLU + MaxPool для каждого размера фильтра
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        # Объединяем результаты
        cat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(cat)

In [5]:
VOCAB_SIZE = sp.GetPieceSize()
model = TextCNN(
    vocab_size=VOCAB_SIZE,
    embed_dim=300,
    n_filters=100,
    filter_sizes= [3,4,5],
    output_dim=3,
    dropout= 0.5,
    pad_idx=0
).to(device)


##### Stats of model:

In [6]:
print(model)
print(f"Параметры: {sum(p.numel() for p in model.parameters())}")

TextCNN(
  (embedding): Embedding(20000, 300, padding_idx=0)
  (convs): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Параметры: 6361203


##### Optimizer and Loss

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

## THE CYCLE 🌀

In [8]:
best_val_loss = float('inf')
best_model_path = 'textcnn_best.pth'


N_EPOCHS = 10

for epoch in tqdm(range(N_EPOCHS), desc="Epoch"):
    model.train()
    
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (tokens, labels) in enumerate(train_loader):
        tokens, labels = tokens.to(device), labels.to(device)
        
        # 1. Обнуляем градиенты
        optimizer.zero_grad()
        
        # 2. Forward
        predictions = model(tokens)
        
        # 3. Loss
        loss = criterion(predictions, labels)
        
        # 4. Backward
        loss.backward()
        
        # 5. Шаг оптимизатора
        optimizer.step()
        
        # Метрики
        train_loss += loss.item()
        _, predicted = predictions.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
    
    if 1==1:
        model.eval()

        test_loss = 0.0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for batch_idx, (tokens, labels) in enumerate(test_loader):
                tokens, labels = tokens.to(device), labels.to(device)

                predictions = model(tokens)
                loss = criterion(predictions, labels)


                test_loss += loss.item()
                _, predicted = predictions.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        test_acc = 100. * test_correct / test_total
        val_loss_avg = test_loss / len(test_loader)
        if val_loss_avg < best_val_loss:
            best_val_loss = val_loss_avg
            torch.save(model.state_dict(), best_model_path)
            print(f'✅ НОВАЯ ЛУЧШАЯ! Val Loss: {val_loss_avg:.4f}')
        print(f'EpochT: {epoch+1:02}, Loss: {test_loss/len(test_loader):.4f}, Acc: {test_acc:.2f}%')
            
    
    # Эпоха закончена
    train_acc = 100. * train_correct / train_total
    print(f'Epoch: {epoch+1:02}, Loss: {train_loss/len(train_loader):.4f}, Acc: {train_acc:.2f}%')

print("✅DONE✅")

Epoch:  10%|█         | 1/10 [03:59<35:55, 239.52s/it]

✅ НОВАЯ ЛУЧШАЯ! Val Loss: 0.6556
EpochT: 01, Loss: 0.6556, Acc: 69.34%
Epoch: 01, Loss: 0.8088, Acc: 62.21%


Epoch:  20%|██        | 2/10 [08:20<33:38, 252.37s/it]

✅ НОВАЯ ЛУЧШАЯ! Val Loss: 0.6227
EpochT: 02, Loss: 0.6227, Acc: 72.07%
Epoch: 02, Loss: 0.6512, Acc: 70.79%


Epoch:  30%|███       | 3/10 [12:52<30:29, 261.31s/it]

...

EpochT: 08, Loss: 1.0377, Acc: 70.23%
Epoch: 08, Loss: 0.2726, Acc: 89.43%


Epoch:  90%|█████████ | 9/10 [40:45<04:35, 275.58s/it]

EpochT: 09, Loss: 1.2088, Acc: 69.53%
Epoch: 09, Loss: 0.2380, Acc: 90.80%


Epoch: 100%|██████████| 10/10 [45:03<00:00, 270.38s/it]

EpochT: 10, Loss: 1.2529, Acc: 69.66%
Epoch: 10, Loss: 0.2182, Acc: 91.62%
✅DONE✅


## Extra data 🗃️

In [9]:
train2_dataset = JsonLDataset('train2.jsonl')
test_dataset = JsonLDataset('test.jsonl')

train2_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

### Cycle №2

In [10]:
model.load_state_dict(torch.load('textcnn_best.pth'))

<All keys matched successfully>

In [11]:
best_val_loss = float('inf')
best_model_path = 'textcnn_best.pth'


N_EPOCHS = 5

for epoch in tqdm(range(N_EPOCHS), desc="Epoch"):
    model.train()
    
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (tokens, labels) in enumerate(train2_loader):
        tokens, labels = tokens.to(device), labels.to(device)
        
        # 1. Обнуляем градиенты
        optimizer.zero_grad()
        
        # 2. Forward
        predictions = model(tokens)
        
        # 3. Loss
        loss = criterion(predictions, labels)
        
        # 4. Backward
        loss.backward()
        
        # 5. Шаг оптимизатора
        optimizer.step()
        
        # Метрики
        train_loss += loss.item()
        _, predicted = predictions.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
    
    if 1==1:
        model.eval()

        test_loss = 0.0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for batch_idx, (tokens, labels) in enumerate(test_loader):
                tokens, labels = tokens.to(device), labels.to(device)

                predictions = model(tokens)
                loss = criterion(predictions, labels)


                test_loss += loss.item()
                _, predicted = predictions.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        test_acc = 100. * test_correct / test_total
        val_loss_avg = test_loss / len(test_loader)
        if val_loss_avg < best_val_loss:
            best_val_loss = val_loss_avg
            torch.save(model.state_dict(), best_model_path)
            print(f'✅ НОВАЯ ЛУЧШАЯ! Val Loss: {val_loss_avg:.4f}')
        print(f'EpochT: {epoch+1:02}, Loss: {test_loss/len(test_loader):.4f}, Acc: {test_acc:.2f}%')
            
    
    # Эпоха закончена
    train_acc = 100. * train_correct / train_total
    print(f'Epoch: {epoch+1:02}, Loss: {train_loss/len(train2_loader):.4f}, Acc: {train_acc:.2f}%')

print("✅DONE✅")

Epoch:  20%|██        | 1/5 [04:38<18:32, 278.13s/it]

✅ НОВАЯ ЛУЧШАЯ! Val Loss: 0.6520
EpochT: 01, Loss: 0.6520, Acc: 71.68%
Epoch: 01, Loss: 0.5805, Acc: 74.65%


Epoch:  40%|████      | 2/5 [08:58<13:23, 267.88s/it]

EpochT: 02, Loss: 0.6641, Acc: 72.16%
Epoch: 02, Loss: 0.5073, Acc: 78.46%


Epoch:  60%|██████    | 3/5 [13:28<08:57, 268.77s/it]

EpochT: 03, Loss: 0.7340, Acc: 71.17%
Epoch: 03, Loss: 0.4387, Acc: 81.78%


Epoch:  80%|████████  | 4/5 [17:55<04:28, 268.13s/it]

EpochT: 04, Loss: 0.7917, Acc: 71.16%
Epoch: 04, Loss: 0.3780, Acc: 84.83%


Epoch: 100%|██████████| 5/5 [22:28<00:00, 269.61s/it]

EpochT: 05, Loss: 0.8848, Acc: 70.37%
Epoch: 05, Loss: 0.3227, Acc: 87.22%
✅DONE✅


## MORE DATA 🗃️🗃️🗃️🗃️🗃️🗃️